In [1]:
import numpy as np
import pandas as pd
from numpyro.diagnostics import summary
from utils.helpers import pickle_load
import matplotlib.pylab as plt

plt.rc("text", usetex=True)
plt.rc("font", **{"family": "sans-serif", "serif": ["Palatino"]})
figSize = (12, 8)
fontSize = 15

In [2]:
ANALYSIS = "des"

if ANALYSIS != "lsst":
    KEYS = [
        "sigma8",
        "Omegac",
        "Omegab",
        "hubble",
        "ns",
        "m1",
        "m2",
        "m3",
        "m4",
        "dz_wl_1",
        "dz_wl_2",
        "dz_wl_3",
        "dz_wl_4",
        "a_ia",
        "eta",
        "b1",
        "b2",
        "b3",
        "b4",
        "b5",
        "dz_gc_1",
        "dz_gc_2",
        "dz_gc_3",
        "dz_gc_4",
        "dz_gc_5",
    ]
else:

    KEYS = [
        "sigma8",
        "Omegac",
        "Omegab",
        "hubble",
        "ns",
        "m1",
        "m2",
        "m3",
        "m4",
        "m5",
        "dz_wl_1",
        "dz_wl_2",
        "dz_wl_3",
        "dz_wl_4",
        "dz_wl_5",
        "a_ia",
        "eta",
        "b1",
        "b2",
        "b3",
        "b4",
        "b5",
        "b6",
        "b7",
        "b8",
        "b9",
        "b10",
        "dz_gc_1",
        "dz_gc_2",
        "dz_gc_3",
        "dz_gc_4",
        "dz_gc_5",
        "dz_gc_6",
        "dz_gc_7",
        "dz_gc_8",
        "dz_gc_9",
        "dz_gc_10",
    ]

# weight
# minuslogpost
# sigma8
# omegac
# omegab
# hubble
# ns
# m1
# m2
# m3
# m4
# m5
# dz_wl_1
# dz_wl_2
# dz_wl_3
# dz_wl_4
# dz_wl_5
# a_ia
# eta
# b1
# b2
# b3
# b4
# b5
# b6
# b7
# b8
# b9
# b10
# dz_gc_1
# dz_gc_2
# dz_gc_3
# dz_gc_4
# dz_gc_5
# dz_gc_6
# dz_gc_7
# dz_gc_8
# dz_gc_9
# dz_gc_10
# minuslogprior
# minuslogprior__0
# chi2
# chi2__LSSTlike


# ---------------------
# weight
# minuslogpost
# sigma8
# omegac
# omegab
# hubble
# ns
# m1
# m2
# m3
# m4
# dz_wl_1
# dz_wl_2
# dz_wl_3
# dz_wl_4
# a_ia
# eta
# b1
# b2
# b3
# b4
# b5
# dz_gc_1
# dz_gc_2
# dz_gc_3
# dz_gc_4
# dz_gc_5
# minuslogprior
# minuslogprior__0
# chi2
# chi2__my_likelihood

In [3]:
def summary_calculation(
    samples1: np.ndarray, samples2: np.ndarray, neval: int
) -> pd.DataFrame:
    record = []
    for i, key in enumerate(KEYS):
        testsamples = np.vstack(([samples1[:, i], samples2[:, i]]))
        summary_stats = summary(testsamples)
        summary_stats[key] = summary_stats.pop("Param:0")
        record.append(summary_stats)

    record_df = []
    for i in range(len(record)):
        record_df.append(
            pd.DataFrame(record[i]).round(3).loc[["r_hat", "n_eff", "mean", "std"]]
        )

    record_df = pd.concat(record_df, axis=1).T
    record_df["scaled_n_eff"] = record_df["n_eff"] / neval
    return record_df

## Cobaya

In [4]:
def cobaya_statistics(engine="jaxcosmo", cov=False):

    record_samples = []
    nsamples = []
    nlike = 0
    for i in range(2):
        if ANALYSIS != "lsst":
            if cov:
                fname = f"CobayaDES/withcov/{engine}_{i+1}/des.1.txt"
                print("DES with covariance")
            else:
                fname = f"CobayaDES/withoutcov/{engine}_{i+1}/output_prefix.1.txt"

        else:
            if cov:
                fname = f"CobayaLSST/{engine}_cov_{i+1}/lsst.1.txt"
            else:
                fname = f"CobayaLSST/{engine}_{i+1}/lsst.1.txt"

        file = np.loadtxt(fname)
        weight = file[:, 0]
        samples = file[:, 2:-4]
        nlike += sum(weight)
        record_samples.append(samples)
        nsamples.append(samples.shape[0])

    min_nsamples = min(nsamples)

    stats = summary_calculation(
        record_samples[0][-min_nsamples:], record_samples[1][-min_nsamples:], nlike
    )

    return stats

In [5]:
## LSST
# 0.0002872206804902297 (Cobaya EH)
# 0.027523081065323297 (NUTS EH)

# 0.00012025138105330559 (Cobaya EMU)
# 0.027563007453589822 (NUTS EMU)

## LSST (with cov for Cobaya)

# 0.02756300693233674 (NUTS EMU)
# 0.003097665090667806 (Cobaya EMU)

# 0.027523081065323297 (NUTS EH)
# 0.003518281352371748 (Cobaya EH)

## DES
# 0.04482185394258569 (NUTS EH)
# 0.004543626076053639 (Cobaya EH)

# 0.03415634511934331 (NUTS Emulator)
# 0.004761498346259732 (Cobaya Emulator)

## DES (with covariance)
# 0.03415634511934331 (NUTS Emulator)
# 0.005597090492950893 (Cobaya Emulator)

# 0.04482185394258569 (NUTS EH)
# 0.0050194485997476805 (Cobaya EH)

In [6]:
# LSST - EH
0.027523081065323297 / 0.0002872206804902297

95.825554825463

In [7]:
# LSST - Emulator
0.027563007453589822 / 0.00012025138105330559

229.21156673761246

In [8]:
# DES - EH
0.04482185394258569 / 0.004543626076053639

9.864776104444681

In [9]:
# DES - Emulator
0.03415634511934331 / 0.004761498346259732

7.173444709095387

With Covariance

In [10]:
# LSST - EH
0.027523081065323297 / 0.003518281352371748

7.822876657311504

In [11]:
# LSST - Emulator
0.027563007453589822 / 0.003097665090667806

8.89799466592681

In [12]:
# DES - EH
0.04482185394258569 / 0.0050194485997476805

8.929637001330946

In [13]:
# DES - Emulator
0.03415634511934331 / 0.005597090492950893

6.102517935409587

## NUTS

In [14]:
def nuts_stats(engine="jaxcosmo"):

    if ANALYSIS == "lsst":
        sampler = pickle_load("lsst", f"nuts_sampler_{engine}")
    else:
        sampler = pickle_load("samples", f"{engine}_nuts_small_ss_high_td")

    nsamples = sampler.num_chains * sampler.num_samples
    num_steps = (
        sampler.get_extra_fields(group_by_chain=True)["num_steps"].sum(1).sum(0).item()
    )
    samples = sampler.get_samples(group_by_chain=True)
    record = []
    for key in KEYS:
        parameter_samples = samples[key]
        summary_stats = summary(parameter_samples)
        summary_stats[key] = summary_stats.pop("Param:0")
        record.append(summary_stats)

    record_df = []
    for i in range(len(record)):
        record_df.append(
            pd.DataFrame(record[i]).round(3).loc[["r_hat", "n_eff", "mean", "std"]]
        )

    record_df = pd.concat(record_df, axis=1).T
    record_df["scaled_n_eff"] = record_df["n_eff"] / num_steps
    return record_df

In [15]:
def nuts_stats_jaxcosmo():
    info = pickle_load("lsst", "nuts_jaxcosmo_info")
    samples, steps = info["samples"], info["steps"]
    num_steps = sum(steps[0]) + sum(steps[1])

    samples_1 = []
    samples_2 = []
    for key in KEYS:
        samples_1.append(samples[key][0])
        samples_2.append(samples[key][1])
    samples_1 = np.asarray(samples_1)
    samples_2 = np.asarray(samples_2)
    df = summary_calculation(samples_1.T, samples_2.T, num_steps)
    return df

In [16]:
def numpyro_model():
    pass

In [17]:
if ANALYSIS == "lsst":
    df_cobaya_emu = cobaya_statistics("emulator", cov=True)
    df_nuts_emu = nuts_stats("emulator")

    df_cobaya_jc = cobaya_statistics("jaxcosmo", cov=True)

    # because we used JAX 0.4.25 on Glamdring - this is separate
    df_nuts_jc = nuts_stats_jaxcosmo()

else:

    df_cobaya_jc = cobaya_statistics(engine="jaxcosmo", cov=True)
    df_nuts_jc = nuts_stats(engine="jaxcosmo")

    df_cobaya_emu = cobaya_statistics(engine="emulator", cov=True)
    df_nuts_emu = nuts_stats(engine="emulator")

DES with covariance
DES with covariance
DES with covariance
DES with covariance


In [18]:
np.abs(
    (df_cobaya_jc["mean"] - df_nuts_jc["mean"])
    / np.sqrt(df_cobaya_jc["std"] ** 2 + df_nuts_jc["std"] ** 2)
)

sigma8     0.022807
Omegac     0.027724
Omegab     0.000000
hubble     0.014428
ns         0.012854
m1         0.031419
m2         0.000000
m3         0.000000
m4         0.000000
dz_wl_1    0.000000
dz_wl_2    0.000000
dz_wl_3    0.000000
dz_wl_4    0.000000
a_ia       0.011560
eta        0.007613
b1         0.027296
b2         0.030628
b3         0.021755
b4         0.026428
b5         0.020794
dz_gc_1    0.000000
dz_gc_2    0.000000
dz_gc_3    0.000000
dz_gc_4    0.000000
dz_gc_5    0.070711
dtype: float64

In [19]:
np.abs(
    (df_cobaya_emu["mean"] - df_nuts_emu["mean"])
    / np.sqrt(df_cobaya_emu["std"] ** 2 + df_nuts_emu["std"] ** 2)
)

sigma8     0.000000
Omegac     0.000000
Omegab     0.000000
hubble     0.000000
ns         0.012854
m1         0.000000
m2         0.000000
m3         0.000000
m4         0.032141
dz_wl_1    0.000000
dz_wl_2    0.000000
dz_wl_3    0.070711
dz_wl_4    0.000000
a_ia       0.011466
eta        0.003389
b1         0.011545
b2         0.005481
b3         0.005819
b4         0.004683
b5         0.000000
dz_gc_1    0.000000
dz_gc_2    0.000000
dz_gc_3    0.000000
dz_gc_4    0.000000
dz_gc_5    0.000000
dtype: float64

In [20]:
df_nuts_emu["scaled_n_eff"].mean()

0.03415634511934331

In [21]:
df_cobaya_emu["scaled_n_eff"].mean()

0.005597090492950893

In [22]:
df_nuts_jc["scaled_n_eff"].mean()

0.04482185394258569

In [23]:
df_cobaya_jc["scaled_n_eff"].mean()

0.0050194485997476805

In [24]:
df_nuts_emu["scaled_n_eff"].mean() / df_cobaya_emu["scaled_n_eff"].mean()

6.102517935409587

In [25]:
df_nuts_jc["scaled_n_eff"].mean() / df_cobaya_jc["scaled_n_eff"].mean()

8.929637001330946

In [26]:
df_nuts_emu.head()

,r_hat,n_eff,mean,std,scaled_n_eff
sigma8,1.0,5983.970,0.840,0.064,0.009935
Omegac,1.0,8488.082,0.229,0.024,0.014093
Omegab,1.0,17761.095,0.043,0.007,0.029489
hubble,1.0,16785.483,0.719,0.051,0.027869
ns,1.0,12301.297,0.957,0.056,0.020424


In [27]:
df_cobaya_emu.head()

,r_hat,n_eff,mean,std,scaled_n_eff
sigma8,1.000,8859.678,0.840,0.062,0.004792
Omegac,1.000,8822.220,0.229,0.023,0.004771
Omegab,1.000,10696.223,0.043,0.007,0.005785
hubble,1.000,9165.396,0.719,0.049,0.004957
ns,1.001,10612.753,0.958,0.054,0.005740


In [28]:
df_nuts_jc.head()

,r_hat,n_eff,mean,std,scaled_n_eff
sigma8,1.0,5089.511,0.828,0.063,0.011469
Omegac,1.0,7251.647,0.229,0.026,0.016341
Omegab,1.0,16921.667,0.045,0.007,0.038133
hubble,1.0,13851.484,0.711,0.050,0.031214
ns,1.0,10428.335,0.958,0.056,0.023500


In [29]:
df_cobaya_jc.head()

,r_hat,n_eff,mean,std,scaled_n_eff
sigma8,1.0,8175.366,0.830,0.061,0.004136
Omegac,1.0,8748.084,0.228,0.025,0.004425
Omegab,1.0,9656.889,0.045,0.007,0.004885
hubble,1.0,10084.430,0.712,0.048,0.005101
ns,1.0,9433.665,0.959,0.054,0.004772
